# Capacity Building
## Prerequisites
- Some basic understanding of Python variables, data types, looping, conditionals and functions will be of benefit
- Completion of  01-basic-model.ipynb, 02-flow-types.ipynb

## A practical application

So far, we've looked at creating a compartmental model, adding flows, requesting derived outputs, and using different solvers and stratifications.<br> Today we take a break from theory to look at the application of these concepts to an epidemic.

## Data inputs
### Imports
Install the summer package, as per previous notebooks

In [ ]:
%pip install summerepi

In [ ]:
# Python standard library imports come first
from datetime import datetime, timedelta
from typing import List

# Then external package imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from summer import CompartmentalModel
from summer import Stratification
from matplotlib import pyplot as plt

# Set pandas to use our favourite interactive plotting tool
pd.options.plotting.backend = "plotly"

# Define constants
GITHUB_MOH = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/"

MOH_FILES = [
    "cases_malaysia",
    "deaths_malaysia",
    "hospital",
    "icu",
    "cases_state",
    "deaths_state",
]

# Define an arbitrary reference date,
# because we will need numbers (not dates) to go into the model solver
COVID_BASE_DATE = datetime(2019, 12, 31)

# Will need this for indexing the MoH data later
region = "Malaysia"

### Utility functions
You can ignore this. Same code as in previous notebooks.

In [ ]:
def fetch_mys_data(base_url: str, file_list: List[str]) -> pd.DataFrame:
    """
    Request files from MoH and combine them into one data frame.
    """
    a_list = []
    for file in file_list:
        data_type = file.split("_")[0]
        df = pd.read_csv(base_url + file + ".csv")
        df["type"] = data_type
        a_list.append(df)
    df = pd.concat(a_list)

    return df

### Get data
Now call the `fetch_mys_data` function and pass it the MoH url.<br>
Also identical to previous notebooks.

In [ ]:
df = fetch_mys_data(GITHUB_MOH, MOH_FILES)

# Same preprocessing steps as in notebook 1
df.loc[df["state"].isna(), "state"] = "Malaysia"
df["date"] = pd.to_datetime(df["date"])
df["date_index"] = (df["date"] - COVID_BASE_DATE).dt.days

# Configure mask for analysis
mask = (df["state"] == region) & (df["type"] == "cases")

# Also download the population data
population_url = (
    "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/static/population.csv"
)
df_pop = pd.read_csv(population_url)
initial_population = df_pop[df_pop["state"] == region]["pop"][0]

In [ ]:
start_date = datetime(2021, 1, 1)  # Define the model's start date
end_date = start_date + timedelta(days=300)  # Define the model's duration

# Numeric representation of the start and end dates
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
notifications_target = df[mask][start_date_int: end_date_int]["cases_new"]

In [ ]:
def plot_comparison(label_a: str, data_a: pd.Series, label_b: str, data_b: pd.Series, title: str):
    df = pd.DataFrame([data_a,data_b]).T
    df['date'] = df.index

    # Visualize the results
    subplot = {"title": title, "xlabel": "Days", "ylabel": "Value"}
    fig = px.line(df,x='date', y=['notifications','cases_new'],title=title)
    fig.show()

## Flows
### Define flows

In [ ]:
# Create a parameters dictionary - we'll reuse this whenever building the model
parameters = {
    "contact_rate": 0.1,
    "progression_rate": 0.1,
    "recovery_rate": 0.1,
    "death_rate": 0.1,
}

## Stratifications
### Define stratifications

In [ ]:
def get_age_stratification() -> Stratification:
    # Create the stratification
    strata=["young", "old"]
    strat = Stratification(name="age", strata=strata, compartments=["S", "E", "I", "R"])

    # Create our population split dictionary, whose keys match the strata
    pop_split = {"young": 0.25, "old": 0.75}

    # Set a population distribution
    strat.set_population_split(pop_split)

    return strat

In [ ]:
def get_severity_strat() -> Stratification:
    # Create a stratification named 'severity', applying to the infectious, which
    # splits that compartment into 'asymptomatic', 'mild' and 'severe'.
    severity_strata = ["asymptomatic", "mild", "severe"]

    # Notice the new argument ["I"] for the compartment parameter.
    severity_strat = Stratification(name="severity", strata=severity_strata, compartments=["I"])

    # Set a population distribution - everyone starts out asymptomatic.
    severity_strat.set_population_split({"asymptomatic": 1., "mild": 0, "severe": 0})
    
    return severity_strat

In [ ]:
# Add an adjustment to the 'progression' flow for young & old people.
young_progression_adjustments = {
    "asymptomatic": 0.5,  # 50% of cases are asympt
    "mild": 0.4,  # 40% of cases are mild
    "severe": 0.1,  # 10% of cases are severe
}

old_progression_adjustments = {
    "asymptomatic": 0.1,  # 10% of cases are asympt
    "mild": 0.4,  # 40% of cases are mild
    "severe": 0.5,  # 50% of cases are severe
}

In [ ]:
all_age_infectiousness_adjustments = {
    "asymptomatic": 0.5,
    "mild": None,
    "severe": 1.5,
}

### Build stratification

In [ ]:
# Get the age stratification
age_strat = get_age_stratification()

# Get the severity stratification
severity_strat = get_severity_strat()

In [ ]:
severity_strat.set_flow_adjustments(
    "progression",
    young_progression_adjustments,
    source_strata={
        "age": "young"
    },  # Only apply this adjustment to flows of young people
)

severity_strat.set_flow_adjustments(
    "progression",
    old_progression_adjustments,
    source_strata={"age": "old"},  # Only apply this adjustment to flows of old people
)

In [ ]:
# Adjust infectiousness levels (for all age groups)
severity_strat.add_infectiousness_adjustments(
    "I",
    all_age_infectiousness_adjustments
)

## Model
### Define a model

Recall the `build_base_model` wrapper function from the last training session.

In [ ]:
def build_base_model() -> CompartmentalModel:
    model = CompartmentalModel(
        times=(start_date_int, end_date_int),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
    )

    model.set_initial_population(distribution={"S": initial_population - 100, "E": 0, "I": 100})

    return model

In [ ]:
def build_model_with_flows(parameters: dict) -> CompartmentalModel:

    # This base model does not take parameters, but have a think about how it might...
    model = build_base_model()

    # Susceptible people can get infected.
    model.add_infection_frequency_flow(
        name="infection", contact_rate=parameters["contact_rate"], source="S", dest="E"
    )
    # Expose people transition to infected.
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover.
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment.
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # Importantly, we will also request an output for the 'progression' flow, and name this 'notifications'
    # This will be available after a model run using the get_derived_outputs_df() method

    model.request_output_for_flow("notifications", "progression")

    return model

### Build the model

In [ ]:
model = build_model_with_flows(parameters)
# Apply stratifications
model.stratify_with(age_strat)
model.stratify_with(severity_strat)

## Outputs

In [ ]:
# Build and run model with no stratifications
#model = build_model_with_flows(parameters)
model.run()

In [ ]:
# Plot compartments
outputs_df = model.get_outputs_df()
outputs_df.plot()

In [ ]:
model_derived_outputs = model.get_derived_outputs_df()

In [ ]:
proportion_of_cases_detected = 0.5

plot_comparison("Notification", 
                notifications_target, 
                "Modelled",
                model_derived_outputs["notifications"] * proportion_of_cases_detected,
                "Modelled vs data")